In [74]:
import os

In [75]:
%pwd

'c:\\Users\\abhay\\Desktop\\Internship\\Data-Engineering-Internship-Bosch'

In [9]:
os.chdir("../")

In [10]:
%pwd

'c:\\Users\\abhay\\Desktop\\Internship\\Data-Engineering-Internship-Bosch'

In [88]:
from dataclasses import dataclass
from pathlib import Path

In [89]:
@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [90]:
from src.dataEngineer.utils.common import *
CONFIG_FILE_PATH = Path("config/config.yaml")


In [91]:
class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:

        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path

        )

        return data_transformation_config



In [92]:
import pandas as pd

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def aggregated_metrices(self):
        try:
            data = pd.read_csv(self.config.data_path)
            data["timestamp"] = pd.to_datetime(
            data["Date"].astype(str) + " " + data["Time"].astype(str),
            dayfirst=True,
            errors="coerce",
            utc=True
            )
            data = data.drop(columns=["Date","Time"])
            numeric_cols = data.select_dtypes(include="number").columns
            aggregated_metrics = data[numeric_cols].agg(["min","max","mean","std"]).T.reset_index()
            aggregated_metrics.columns = ["feature","min","max","avg","std"]
            aggregated_metrics["source_file"] = "AirQuality.csv"
            aggregated_metrics["source_system"] = "kaggle_airquality_uci"
            aggregated_metrics["current_time"] = pd.Timestamp.now(tz="UTC")
            aggregated_metrics.to_csv("aggregated_metrics.csv", index = False)

        except Exception as e:
            raise e

        return aggregated_metrics

    
        
        

IndentationError: expected an indented block (2718619934.py, line 26)

In [109]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.aggregated_metrices()
except Exception as e:
    raise e

[2025-08-17 08:08:18,100: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-17 08:08:18,105: INFO: common: created directory at : artifacts]
[2025-08-17 08:08:18,107: INFO: common: created directory at : artifacts/data_transformation]


In [ ]:
df = pd.read_csv("artifacts/data_ingestion/AirQualityUCI.csv")
df


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10-03-2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10-03-2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10-03-2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10-03-2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10-03-2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df["Date"].dtype

dtype('O')

In [51]:
numeric_cols = df.select_dtypes(include="number").columns
numeric_cols

Index(['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)',
       'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)',
       'T', 'RH', 'AH', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [60]:
df["timestamp"] = pd.to_datetime(
    df["Date"].astype(str) + " " + df["Time"].astype(str),
    dayfirst=True,
    errors="coerce",
    utc=True
)
aggregates = df[numeric_cols].agg(["min","max","mean","std"]).T.reset_index()
aggregates.columns = ["feature","min","max","avg","std"]
aggregates["source_file"] = "AirQuality.csv"
aggregates["source_system"] = "kaggle_airquality_uci"
aggregates["sensor_data(from)"] = df["timestamp"].min()
aggregates["sensor_data(till)"] = df["timestamp"].max()
aggregates

,feature,min,max,avg,std,source_file,source_system,sensor_data(from),sensor_data(till)
0,CO(GT),-200.0,11.900,-34.207524,77.657170,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
1,PT08.S1(CO),-200.0,2040.000,1048.990061,329.832710,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
2,NMHC(GT),-200.0,1189.000,-159.090093,139.789093,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
3,C6H6(GT),-200.0,63.700,1.865683,41.380206,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
4,PT08.S2(NMHC),-200.0,2214.000,894.595276,342.333252,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
5,NOx(GT),-200.0,1479.000,168.616971,257.433866,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
6,PT08.S3(NOx),-200.0,2683.000,794.990168,321.993552,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
7,NO2(GT),-200.0,340.000,58.148873,126.940455,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
8,PT08.S4(NO2),-200.0,2775.000,1391.479641,467.210125,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00
9,PT08.S5(O3),-200.0,2523.000,975.072032,456.938184,AirQuality.csv,kaggle_airquality_uci,2004-03-10 18:00:00+00:00,2005-04-04 14:00:00+00:00


In [58]:
aggregates.to_csv("aggregates.csv", index = False)